# Data Exploration

Here we will not treat the binaries like a black box bot rather use our knowledge of machine language in general
and particular ISA's to analyze the data set.
We will gather some general features common to most or all ISA's and deep dive into a few.

Online disassembler with > 60 archs
https://onlinedisassembler.com/static/home/index.html

https://en.wikibooks.org/wiki/X86_Disassembly/Disassemblers_and_Decompilers#Linux_Disassemblers

## Questions
###1
Is it possible to have a binary snippet from the game represent a valid machine instruction in two distinct ISA's

Experiment: Using https://onlinedisassembler.com/odaweb/Ufx0uq75/0 Select arch = alphav5 or alphav6 (correct target)
00 00 00 00 00 01 7a 52 00 04 78 1a 01 1b 0d 1e 00 18 00 00 00 18 00 00 00 00 00 00 00 8c 00 00 00 00 43 0e 10 41 9a 02 5e da 0e 00 00 03 00 e3 43 00 00 3d 24 00 00 61 b0 00 00 42 20 04 00 42
Output has no "unknown" translations
Endianness does not affect output
Select arm
Output has no "unknown" translations
i386-x86_64:
  27 lines
  unusual:
    0e: (bad)
    1b0d1e001800: apparently 6 word instruction?
mips:
  unusual: several items are just copied from hex string to assembly.  Perhaps the IDK for this arch?
  notes: all lines are 8 hex chars
  029a4110: 0x29a4110
mips16:
  unusual: three jal<x> instructions of 8 hex chars, the rest are 4 hex chars.
sh4:
  notes: all lines are 4 hex chars
  unusual:
    0018: sett; single sintruction no args?
  

# sparc

Overview http://moss.csc.ncsu.edu/~mueller/codeopt/codeopt00/notes/sparc.html
## RISC
### 3 types of instruction
#### ALU + shift
opcode reg1,reg2,reg3    !reg1 op reg2    -> reg3
opcode reg1,const13,reg3 !reg1 op const13 -> reg3
#### Load, Store
opcode [reg1+reg2],reg3
opcode [reg1+const13],reg3
#### Branch
opcode address


32 registers, 32 fp registers, 32 coprocessor registers, 10 special registers, 4 unary ops, 

All load and store begin with 11



Examples
http://www.cs.unm.edu/~maccabe/classes/341/labman/node9.html
Example: Hand assemble the instruction:
ldd     [%r4+%r7], %r11
Because this instruction uses two registers in the address specification, it is encoded using the first format shown in Figure 9.1. As such, we must determine the values for the rd, op tex2html_wrap_inline5419 , rs tex2html_wrap_inline5421 , and rs tex2html_wrap_inline5423 fields. The following table summarizes these encodings:

tabular2256

These encodings lead to the following machine instruction:

picture2263

That is, 1101 0110 0001 1001 0000 0000 0000 0111 in binary, or 0xD6190007.

If the assembly language instruction only uses a single register in the address specification (e.g., register indirect addressing), the register is encoded in one of the source register fields (i.e., sr tex2html_wrap_inline5421 or sr tex2html_wrap_inline5423 ) while %r0 is encoded in the other. It doesn't matter which field holds the register specified in the assembly language instruction and which field holds the encoding for %r0. However, isem-as encodes %r0 in sr tex2html_wrap_inline5423 .

Example: Hand assemble the instruction:
ldub     [%r23], %r19
Because this instruction uses registers in the address specification, it is encoded using the first format shown in Figure 9.1. As such, we must determine the values for the rd, op tex2html_wrap_inline5419 , rs tex2html_wrap_inline5421 , and rs tex2html_wrap_inline5423 fields. The following table summarizes these encodings:

tabular2311

These encodings lead to the following machine instruction:

picture2318

That is, 1110 0110 0000 1101 1100 0000 0000 0000 in binary, or 0xE60DC000.

# ARM

32 bits
instruction type, condition, operands

some of the status bits are stored in bits 26..31 of R15

### Groupings 5.5

Group one

MOV, ADD etc. 1 s-cycle. If <rhs> contains a shift count in a register (i.e. not an immediate shift), add 1 s-cycle. If <dest> is R15, add 1 s + 1 n-cycle.

Group one A

MUL, MLA. 1 s + 16 i- cycles worst-case.

Group two

LDR. 1 s + 1 n + 1 i-cycle. If <dest> is R15, add 1 s + 1n-cycle.

STR. 2 n-cycles.

Group three

LDM. (regs-1) s + 1 n + 1 i-cycle. Regs is the number of registers loaded. Add 1 s + 1 n-cycles if R15 is loaded.

STM. 2 n + (regs-1) s-cycles.

Group four

B, BL. 2 s + 1 n-cycles.

Group five

SWI. 2 s + 1 n-cycles.

Hex to ARM and back converter
http://armconverter.com/

All arithmetic functions have opcode 0x -> 000000


https://stackoverflow.com/questions/11785973/converting-very-simple-arm-instructions-to-binary-hex

mov r0, r0 should go like this:

1110 00 0 0 1101 0000 0000 00000000

# MIPS

### Groupings 5

https://en.wikibooks.org/wiki/MIPS_Assembly/Instruction_Formats

Converter assembly <-> hex/binary https://www.eg.bucknell.edu/~csci320/mips_web/


In [0]:
opcode_mapping = """
R:
 description: all the data values used by the instruction are located in registers.
 opcode_pattern: 
  opcode: 
    bits: 6
  rs: 
    bits: 5
    indices: [21,25]
  rt: 
    bits: 5
    indices: [16, 20]
  rd: 
    bits: 5
  shift: 
    bits: 6
  funct: 
    bits: 6

I:
  description: the instruction must operate on an immediate value and a register value. 
  opcode_pattern:
    opcode: 
      bits: 6
    rs: 
      bits: 5
    rt: 
      bits: 5
    IMM: 
      bits: 16
J:
  description: jump instructions
  opcode_pattern:
    opcode: 
      bits: 6
    pseudo_addr: 
      bits: 26
      description: 
       - he two least significant bits are removed, and the 4 most significant bits are removed, 
       - and assumed to be the same as the current instruction's address.
      
FR:
  description: instruction uses data only from floating-point registers
 opcode_pattern: 
  opcode: 
    bits: 6
  fmt: 
    bits: 5
  ft: 
    bits: 5
  fs: 
    bits: 5
  fd: 
    bits: 6
  funct: 
    bits: 6  
  
FI:
  description: the instruction must operate on an immediate value and a register value.
  opcode_pattern:
    opcode: 
      bits: 6
    fmt: 
      bits: 5
    ft: 
      bits: 5
    IMM: 
      bits: 16    
"""



# x86_64

assembly <-> hex https://defuse.ca/online-x86-assembler.htm

Special
Stack Pointer, Base Pointer

In [0]:
base64_encoded = u'GAAAABgAAAAAAAAAqAAAAABIDhBHmgJa2g4AAAAAQaAAAD0kAABetxEE/0cCAOJDAAAQIgAAQbAAAF0kAABCoA=='

# SH4
http://www.shared-ptr.com/sh_insns.html

In [0]:
import base64
import binascii

In [0]:
hx = base64.b64decode(base64_encoded)
repr(hx)

In [0]:
len(hx)

In [0]:
hx[0]

In [0]:
binascii.hexlify('00=$')

In [0]:
bins = [bin(long(binascii.hexlify(x), 16)) for x in hx]
print(bins[0:6])

In [0]:
bins = [x + '0'*(2 + 8 - len(x)) for x in bins]
print(bins[0:6])

In [0]:
len(bins)/4

In [0]:
2**32/10**6

In [0]:
import etl

In [0]:
data_set = etl.load('ml_challenge/4096_512_512_6f9ef7d8.json')

In [0]:
orig_X_train, orig_Y_train, orig_train_targets = data_set['train']['binary_data'], data_set['train']['answers'], data_set['train']['targets']
orig_X_dev, orig_Y_dev, orig_dev_targets = data_set['dev']['binary_data'], data_set['dev']['answers'], data_set['dev']['targets']
orig_X_test, orig_Y_test, orig_test_targets = data_set['test']['binary_data'], data_set['test']['answers'], data_set['test']['targets']

In [0]:
print(orig_X_train[0:6])
print(orig_Y_train[0:6])
print(orig_train_targets[0:6])

In [0]:
hex_X_train, train_misfits = etl.hex_data(orig_X_train)
hex_X_dev, dev_misfits = etl.hex_data(orig_X_dev)
hex_X_test, test_misfits = etl.hex_data(orig_X_test)

In [0]:
print(hex_X_train[0:6])

In [0]:
import pandas as pd

In [0]:
dft = pd.DataFrame(zip(hex_X_train, orig_Y_train))

In [0]:
dft.describe()

Observe that there are 4096 unique snippets, so no collisions between archs here.  
There are 16 ** (len(snippet) * (2 digits per 'word')) = 16 ** 128 possibilities

In [0]:
dft.columns = ['hex_list', 'arch']

In [0]:
dft.head()

For each ISA find the frequencies of 2-char 'words' like '4d', 'cf'

In [0]:
import collections
from __future__ import division

In [0]:
def build_counts(hex_lists, archs):
    """
    hex_lists: list of 64 item list of 32-bit hex items
    archs: list of arch strings
    @returns: 12 counters one for each arch
    """
    counters = dict(zip(etl.SUPPORTED_ARCHITECTURES, [collections.Counter() for 
                                                      i in range(len(etl.SUPPORTED_ARCHITECTURES))]))
    for h, arch in zip(hex_lists, archs):
        counters[arch].update(h)
    return counters
        

In [0]:
counters = build_counts(hex_X_train, orig_Y_train)
print(counters.keys())

In [0]:
def freq_stats(counters):

    stats = {}
    for arch in etl.SUPPORTED_ARCHITECTURES:
        total_instructions = 0
        unique_instructions = 0
        repeated_instructions = 0
        repeats = []
        for instruction, count in counters[arch].most_common():
            total_instructions += count
            unique_instructions += 1
            if count > 1:
                repeats.append(instruction)
                repeated_instructions += 1
        stats[arch] = {'vocab_size': unique_instructions,
                       'repeated_instructions': repeated_instructions,
                       #'repeats': set(repeats),
                       'unique_instructions': unique_instructions,
                       'total_instructions': total_instructions,
                       'percent_repeated': (repeated_instructions/unique_instructions)*100.0,
                       'most_common_20': set([x[0] for x in counters[arch].most_common(20)]),
                       'least_common_20': set([x[0] for x in counters[arch].most_common()[:-20-1:-1]])}
    return stats
                
        

In [0]:
fstats = freq_stats(counters)
print(fstats)

Conclusion: At the granularity of 32-bit instructions only ~ 1% of instructions are in the repetition set of an arch.
The total instructions seen by each counter is total (samples/num_archictectures) * (instructions/sample)

In [0]:
4096/12 * 64

In [0]:
def stop_words(freq_stats):
    union = set([])
    wildspace = {}
    # could choose any arch to initialize intersection
    intersection = set(freq_stats['sparc']['most_common_20'])
    for arch in freq_stats:
        intersection.intersection(freq_stats[arch]['most_common_20'])
        others_union = set([])
        union = union.union(freq_stats[arch]['most_common_20'])
        others = set(freq_stats.keys()) - set([arch])
        for other in others:
            others_union = others_union.union(freq_stats[other]['most_common_20'])
        wildspace_item = freq_stats[arch]['most_common_20'].difference(others_union)
        wildspace[arch] = wildspace_item
    return intersection, wildspace

In [0]:
stop_words, wildspace = stop_words(fstats)

In [0]:
print(len(stop_words), stop_words)

In [0]:
print(wildspace)

In [0]:
{ orig_Y_train[i]: " ".join(hex_X_train[i]) for i in range(5)}